# Xarray - more than Pandas in multiple dimensions

Ondřej Grover - PyData Prague 5.10.2020

*Ph.D. candidate at the Institute of Plasma Physics of the CAS*

*not a core xarray developer, but an active user and extension developer* 

### Outline

- Motivation: 3 laments of NumPy (and Pandas)
- `xarray` API basics
  - technical structure overview
  - basic usage examples

- More advanced API usage
  - Convenient wrappers: `xrscipy` and `xrrandom` examples
  - Implicit, parallelized `for` loops with Dask broadcasting magic

### If only NumPy slicing and indexing could use labels/coordinates ...

In [ ]:
import numpy as np

x = np.linspace(0, 5)
y = x**2
# how to select y for which 2 < x < 3 ?
# common workaround: array of bool mask
y_sel = y[(2 < x) & (x < 3)]   # not very efficient
y_sel

In [ ]:
import pandas as pd

ys = pd.Series(y, index=x)
ys.loc[2:3]   # efficient and concise

Pandas covers this, **but** is mostly oriented at **1D** + "**2D**" *(`DataFrame` ~ `dict` of `Series` - not as general)*

*(or "flat-ND" with `MultiIndex` - useful with sparse ND data)*

**Xarray** uses and generalizes *Pandas* indexing machinery into *multiple-dimensions*

### If only NumPy had named dimensions ...

*algebraic* dimensions (rows, columns, depth, ...)  **vs.** *physical* dimensions ($x$, $y$, $z$, $t$, ...)

In [ ]:
A = np.random.rand(5, 7, 3)   # tensor of shape (5,7,3)  ... (t, x, y)
# some calculations or saving/loading, reshaping....
B = A.T.swapaxes(2,1)    # can you guess what the shape is ???
x_mean = np.mean(B, axis=?x?)  # where is the x "dimension" ???
x_mean.shape, B.shape

correct dimension axis is 2

**Xarray** assigns names to dimensions ... `axis=N` -> `dim='name'`   (in wrapped functions/methods)
 - no need to remember dimension ordering
 - if dimension missing : `dim='name'` fails explicitly **vs.** `axis=N` might pack a surprise for later ...
 
 *Pandas 2D `DataFrame` has algebraic `index/columns` dimensions, with `MultiIndex` has "dimension"-level names with given order*

### If only NumPy broadcasting magic had named dimensions ...

NumPy broadcasting: implicit (C-fast) `for` loops in:
 - arithmetic operations vectorization
 - universal functions (*ufunc*) arguments vectorization

But not so readable and easy to use  in practice ...

In [ ]:
t = np.linspace(0, 1, 5)   # time array
C = B * np.exp(-t)
C.shape

Must add `.reshape((-1,1))`

**Xarray** handles broadcasting according to dimensions names
 - checks if dimension sizes match (and/or aligns by associated coordinate indices if available)
 - reshaping, transposing, ... for arithmetic operations
 - handles *ufunc* broadcasting via `apply_ufunc` helper API
 

- *with Pandas `MultiIndex` "flat-ND" alignment is still possible*
 
- *PyTorch 1.3 introduced (experimental) "Named Tensors" which track dimensions*

### Enter `xarray`

<div>
<img src="https://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" alt="xarray logo" style="float:left; width:49%;"><img src="https://xarray.pydata.org/en/stable/_images/numfocus_logo.png" alt="NumFOCUS logo" style="float:left; width: 49%;">
</div>

    
- pure Python with (optional) compiled dependencies (NumPy, Pandas, NetCDF, ...)
- superb, well-structured documentation with many examples: [xarray.pydata.org](https://xarray.pydata.org/en/stable/index.html)



Brief historical context:
- evolution of an internal tool developed at The Climate Corporation 
- released as open source in May 2014 as `xray`, renamed in January 2016
- became a fiscally sponsored project of NumFOCUS in August 2018


### Semi-internal `Variable` API: core building-block

- **Wraps** NumPy(-like ... *duck array*) with **dimensions** (+ other optional) *metadata*
- **not** an `ndarray` subclass, but many *duck array* mixin properties (`[]`, `.shape`, `.dtype`, ...)
- inspired by NetCDF file format (v4: ~ HDF5 + geoscience data conventions)

In [ ]:
import xarray as xr    # "canonical" namespace short-hand

data = np.random.rand(2, 2, 2)
dvar = xr.Variable(('t', 'x', 'y'), data,
                   attrs={'simulation': 'random Normal'}, encoding={})   # optional arbitrary metadata
dvar

`DataArray`, `Dataset` containers: API built on *mappings* (~`dict`) of `'name'` -> `Variable`-like 

### The `DataArray` container:  "`Variable` with associated coordinates"
- wraps 1 main array  + (optionally) sets `.name`
- adds `.coords` mapping of *coordinate* name ->  associated coordinate/label `DataArray`s 

In [ ]:
coord = xr.DataArray([], name='not in dims')   # to skip the next code

In [ ]:
data = np.random.rand(2, 2, 3)
darr = xr.DataArray(data, dims=('t', 'x', 'y'),
                    name='smth', coords={'t':[0.2, 0.3],
                                'x': np.arange(2)},
                    attrs={'simulation':'random N(0,1)'})
darr

In [ ]:
# PSEUDOCODE for each coord DataArray in darr.coords
if coord.name in darr.dims and coord.ndim == 1:
    darr.indexes[coord.name] = pd.Index(coord)

In [ ]:
darr.coords['t']

In [ ]:
darr.indexes['x']

### `repr` & HTML representation of  dimensions w&w/o index
 HTML display is default in Jupyter Notebook as of version 0.15.1.
 
|dimensions |HTML| `repr`|
|:-|:-|:-|
|*with* index|**bold**|`*` symbol in `.coords`|
| *without* index| normal| listed explicitly|

In [ ]:
darr.coords['x2'] = darr.coords['x'] - 2
darr

In [ ]:
print(darr)

||Pandas | xarray|
|:-|:-|:-|
|**core building block**|  `Index`| dimensions (`Variable`)|
|**`Index` presence**|  mandatory| optional|

### The `Dataset` container: "mapping of `DataArray`s"
- like `DataFrame` is a mapping of `Series`
- shared **union** of `.coords` of all contained variables

In [ ]:
ds = xr.Dataset({'smth': (['t', 'x', 'y'], data),
            'difrnt': (['y', 'x'], data[0].T)},
        coords={'t': [0.3,0.2], 'x': [0,1]})
ds

- `.data_vars`: name -> `DataArray`
  - used by `Dataset` mapping API (`[]`, `.keys()`, `.items()`, `iter()`, ...) as of v0.11
- `.variables`: union of `.coords` and `.data_vars` -> `(Index)Variable`



||`.coords`|`.data_vars` |
|-:|-:|-:|
|intended for quantities | fixed/independent| measured/dependent |
|indexing usage| yes if 1D| no|
|`.plot` usage|x(,y) values/labels | y,(color) value | 

### Tutorial dataset: air temperature above the US 2013-2014
- **Disclaimer**: I'm not a climate scientist -> just for illustration!

In [ ]:
ds = xr.tutorial.open_dataset('air_temperature')   # downloads+caches ~17 MB NetCDF, wraps xr.open_dataset
ds

### Lazy/on-demand data loading by `open_dataset` (and `open_dataarray`)
- `xr.open_dataset` loads only `.coords` into memory -> selection/slicing by index works!
- data variables loaded only on actual values access (calculation, slicing, ...) or with `.load()`
- `xr.load_dataset` follows `xr.open_dataset` by `.load()` -> loads all data into memory

In [ ]:
airT = ds['air'] # let's start with DataArray
airT

In [ ]:
partyT = airT.loc['2014-01-01']
partyT

### Attribute access aliases
For valid Python attribute names not aliased by class attributes/methods `ds.<name>` tries:
1. `ds['<name>']` which tries in turn
    1. `ds.data_vars['<name>']`
    2. `ds.coords['<name>']`
2. `ds.attrs['<name>']`

In [ ]:
assert ds.air is not ds['air']   # different DataArray instances created on-demand
xr.testing.assert_equal(ds.air, ds['air'])  # but equal in terms contents
xr.testing.assert_equal(ds.lon, ds.coords['lon'])
assert ds.title == ds.attrs['title']

Similar for `DataArray`, but no `.data_vars`

In [ ]:
xr.testing.assert_equal(airT.time, airT.coords['time'])
xr.testing.assert_equal(airT['lon'], airT.coords['lon'])  # yes, this actually works
assert airT.units == airT.units

### Tab-completion support 
- Mixin class implements `._ipython_key_completions_()` based on member sources

|completion features |attribute| item|
|-:|-:|-:|
|data members suggested| with valid Python identifier | all |
|suggestions also shown| class methods/attributes (alias conflicts) | no (+- Jupyter issues)|
|chained tab-completion* | better | worse |

\* Depends on completion library settings.

In [ ]:
ds.air.all

In [ ]:
ds['a'].a

### Basics of `DataArray` NumPy-like API  (shared with `Variable`)
Underlying data (property - checks dimension shape and size on assignment) attributes
- `.data`: the actual data (`np.ndarray` or lazy-data wrapper or Dask or sparse)
- `.values` always return `ndarray` (like `np.asarray`) ... "safer" for function application

In [ ]:
airT.data is airT.values  # True if ndarray and loaded into memory

`ndarray`-like (*duck array*) attributes: `.ndim`, `.shape`, `.size` `.nbytes`, `.dtype`, ... complemented by

- `.dims`: dimension names corresponding to algebraic dimensions
- `.sizes`: mapping of dimensions name -> size  (*likely ordered as `.dims` on Python 3*)

In [ ]:
airT.dims

In [ ]:
airT.sizes

### Indexing and selecting data *when dimensions order is known*
- For `DataArray`  integer indexing by `darr[...]` and label-based indexing by `darr.loc[...]`
- label-based slicing is **end-inclusive** (like Pandas)

In [ ]:
airT[0,50::-2,5]

In [ ]:
airT.loc['2014-02-17 00',10::-2,280]

Copy returned for "fancy indexing" (not representable by integers and/or slices) - otherwise: view 

### Indexing and selecting data by dimension name

- by integers `darr[dict(x=val,...)]`, label-based: `darr.loc[dict(x=val,...)]` 
    - can be used for assignment
- alternatively `darr.isel(x=val,...)` and `darr.sel(x=val,...)`, respectively
   - cannot be used for assignment

- [PEP 472](https://legacy.python.org/dev/peps/pep-0472/): `darr[lat=20:30,time='2014']` 
- requires slice specification by `slice(...)`
- works also for `Dataset`
- vectorized indexing: e.g. point-wise new dimension

In [ ]:
# euivalent to airT.loc['2014-02-17 00',10::-2,280]
airT.sel(lon=280,time='2014-02-17 00',
         lat=slice(10,None,-2))

### Nearest neighbor lookups
- `.sel()`: optional `method='nearest'` (`'ffill'` and `'bfill'` also available)
- optional `tolerance=` for "too far" matches -> `KeyError`

In [ ]:
ds.sel(lat=75.5, method='nearest')

### Matplotlib PyPlot wrappers in `.plot` accessor
- Auto-magic `.plot()` call selects method based on dimensionality
- axis labels from `description` and `units` attributes (if available)

In [ ]:
airT.plot();   # more than 2D -> histogram

In [ ]:
airT.isel(time=500).plot()  # 2D -> pcolormesh

Plots can be further adjusted by matplotlib calls (**after** xarray call - sets title, labels, ...)

### Automatic plot orienting, faceting and grouping by dimension

In [ ]:
airT.sel(time='2014-04-15').plot.pcolormesh(x='lon', col='time')   # 3D array faceted to 2D

In [ ]:
airT.sel(lon=280, lat=slice(23,10)).plot.line('.--', x='time', alpha=0.3, aspect=3, size=4);   # 2D array -> group of 1D lines

### More interactive alternative: `.hvplot` accessor 
 `.plot`-like wrapper of HoloViews (Bokeh)

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import hvplot.xarray    # registers the .hvplot accesor

airT.sel(lat=[20,40,60]).hvplot.line(x='time', groupby='lon', by='lat')

### NumPy methods operating along dimensions
- most NumPy-like methods have an extra `dim='name'` argument -> `axis=N` (still available)
- by default `nan` are skipped in calculations (controlled `skipna=`)
- attributes may be dropped (controlled by `keep_attrs=`)

In [ ]:
np.set_printoptions(edgeitems=1)   # to fit on slide
ds['muT'] = airT.mean(dim='lon') # "data namespace"
ds.muT

In [ ]:
ds

### Wrapping other NumPy methods
- NumPy functions (`ufunc`,...) respecting `__array*` interface will wrap result in `DataArray`
- `axis=darr.get_axis_num('<dim name>')` also possible, but `apply_ufunc` recommended
- arithmetic ops: reshape and align to *inner join* of coords (configurable by `xr.set_options`)

In [ ]:
muT2 = np.sqrt(ds.muT).sel(lat=[20,30,60])  # still a DataArray
dT2 = airT - muT2   # reshapes muT and aligns both to common lat index
dT2

### Warning: Dataset always aligns all members by *outer join*

In [ ]:
ds['dT2'] = dT2    # will fill other lattitudes by nan
ds

May lead to many `nan` values in "sparse" variables

In [ ]:
del ds['dT2']   # luckily we can fix that

### General (g)`ufunc`-like function wrapping by `xr.apply_ufunc()`

1. prepares data in  passed `xarray` containers (including `Dataset`) for a NumPy-aware function 
  - moves **core** dimensions to be *last*, will not be broadcast
  - reshaping, aligning for broadcasting
2. applies function to `ndarray`s
  - expected to apply on **core** dims or all
  - optionally with `np.vectorize`
  - possibly parallelized over Dask chunks
  - great with `numba.(gu)vectorize`
3. wraps result(s) with xarray metadata 

In [ ]:
from scipy.ndimage import sobel    # sobel filter

ds_sobel = xr.apply_ufunc(sobel, ds, 
               input_core_dims=[['lat']],
               output_core_dims=[['lat']],
               kwargs=dict(axis=-1)) 

ds_sobel.air.isel(time=500).plot(x='lon')

### Example SciPy wrappers from `xrscipy`
 - Includes many common SciPy wrapper for FFT, spectral analysis, filtering, integration
 - [xr-scipy.readthedocs.io](https://xr-scipy.readthedocs.io/)


In [ ]:
import xrscipy
import xrscipy.signal as xsignal

# quadratic Savitzky-Golay deriv filter
# over a window of 30 longitudes
dlon = xsignal.savgol_filter(airT, 30, 2, 1, dim='lon')
dlon.isel(time=500).plot()

![SG](https://upload.wikimedia.org/wikipedia/commons/8/89/Lissage_sg3_anim.gif)

### Monte-Carlo-like probability propagation with `xrrandom`
- wraps `scipy.stats` and `arch` (bootstrapping)
- likely to be merged into `xrscipy`
- supports "virtual sample counts" via Dask
- https://github.com/smartass101/xr-random

In [ ]:
import xrrandom
from xrrandom import stats as xstats


mu = airT.mean(dim='time')
sd = airT.mean()   # over all -> 0D
# automatic broadcasting of sampel size
samples = xstats.norm.rvs(loc=mu, scale=sd, samples=1000)
samples

In [ ]:
res = np.log(samples**2).mean(dim='sample')
res

### Dask chunking of `xarray` containers
- `.chunk(<mapping dim -> chunksize>)` turns `.data` into a `DaskArray`
- most methods and operations will work and be added to Dask tree
- `.compute()` evaluates the Dask tree

In [ ]:
from dask.diagnostics import ProgressBar

dairT = airT.chunk({'time':50}) # parallelize in lat
dairT

In [ ]:
with ProgressBar():
    muT =  dairT.mean(dim='time').compute()
muT

### Combining broadcasting with Dask -> parallelized out-of-memory for loops

- Problem: `for` each `time` calculate percentage of times the temperature was warmer

- Solution: create a new array `time2` dimension and broadcast (with chunks) against it

In [ ]:
airT2 = airT.rename(time='time2')
warmer2 = dairT > airT2
warmer2 

In [ ]:
with ProgressBar():
    warmer = warmer2.mean(dim='time2').compute()
warmer

### Many other things not covered (but similar with Pandas or domain specific)

Go check out the docs at [xarray.pydata.org](https://xarray.pydata.org/en/stable/index.html)
- stacking (flattening) dimensions into `MultiIndex`
- group-by (split-apply-combine) operations
- rolling windows, coarsening, weighted averaging
- combining (`xr.concat`) data
- interpolating data
- polynomial fitting
- squeezing, expanding, reordering dimensions
- IO backends
- registering accessors (composition over inheritance)
- ...

### Summary: why you might want to use `xarray`
- explicit **dimensions** metadata
  - more readable and safer code
  - convenient handle on broadcasting magic and function application
- generalizes tried and trusted NumPy and Pandas API  to Dask and beyond
- versatile and quite universal data structures - standardization
  - enable transfer of [extensions](https://xarray.pydata.org/en/stable/related-projects.html) to many different fields


Talk available as Jupyter Notebook at GitHub [smartass101/xarray-pydata-prague-2020](https://github.com/smartass101/xarray-pydata-prague-2020)

*presented with the [RISE](https://rise.readthedocs.io/en/stable/) and  [Split Cell](https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/splitcell/readme.html) extensions*